In [1]:
from importlib import reload
import sys
import util
reload(util)
data = util.FontData
FontDataset = util.FontDataset
import experiment
reload(experiment)
from sklearn import linear_model, neural_network

In [2]:
data.load()

Loading embeddings...done
Loading typographic + semantic vectors...done
Loading images...done
Loading glyphs...done
Loading SVGs...done


In [3]:
train_dataset = FontDataset(data, 'train')
val_dataset = FontDataset(data, 'val')

In [4]:
train_typographic = data.get_all_typographic('train')
val_typographic = data.get_all_typographic('val')

In [5]:
train_image = data.get_all_image('train')
train_image = train_image.reshape(train_image.shape[0], -1)
val_image = data.get_all_image('val')
val_image = val_image.reshape(val_image.shape[0], -1)

In [6]:
train_semantic = data.get_all_semantic('train')
val_semantic = data.get_all_semantic('val')

### Linear Regression: Image -> Typographic

In [7]:
image_typographic_reg = linear_model.LinearRegression()

In [8]:
image_typographic_reg.fit(train_image, train_typographic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [9]:
train_stats, train_typographic_pred = experiment.run_experiment(image_typographic_reg, train_image, train_typographic)

{'RMSE': 1.3977822845845367e-06, 'R2': 0.9999999998341788}


In [12]:
val_stats, val_typographic_pred = experiment.run_experiment(image_typographic_reg, val_image, val_typographic)

{'RMSE': 0.10177131742238998, 'R2': 0.5893038248632995}


### Linear Regression: Typographic -> Semantic

In [13]:
typographic_semantic_reg = linear_model.LinearRegression()

In [14]:
typographic_semantic_reg.fit(train_typographic, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [15]:
train_stats, train_semantic_pred = experiment.run_experiment(typographic_semantic_reg, train_typographic, train_semantic)

{'RMSE': 0.10326619446277618, 'R2': 0.5034361427743564}


In [16]:
val_stats, val_semantic_pred = experiment.run_experiment(typographic_semantic_reg, val_typographic, val_semantic)

{'RMSE': 0.106668621301651, 'R2': 0.5101831289972383}


### Linear Regressions: Image -> Typographic -> Semantic

In [17]:
class SequentialLinearRegression:
    def __init__(self, reg1, reg2):
        self.reg1 = reg1
        self.reg2 = reg2
    def predict(self, X):
        return self.reg2.predict(self.reg1.predict(X))

image_typographic_semantic_reg = SequentialLinearRegression(image_typographic_reg, typographic_semantic_reg)

In [18]:
train_semantic

array([[0.33920993, 0.24672677, 0.45607575, ..., 0.46512891, 0.59643258,
        0.54990847],
       [0.49424582, 0.46839964, 0.37099306, ..., 0.65840484, 0.43551042,
        0.48553453],
       [0.24767428, 0.07544883, 0.25545198, ..., 0.65944604, 0.37622461,
        0.45430069],
       ...,
       [0.73453555, 0.70904671, 0.56249398, ..., 0.47190857, 0.76819777,
        0.45068339],
       [0.74848685, 0.62451316, 0.45640533, ..., 0.50166912, 0.41377995,
        0.35790375],
       [0.71824549, 0.84435252, 0.47846678, ..., 0.72807055, 0.45456135,
        0.31400816]])

In [19]:
train_stats, train_semantic_pred = experiment.run_experiment(image_typographic_semantic_reg, train_image, train_semantic)

{'RMSE': 0.1032661497592926, 'R2': 0.503436530931538}


In [20]:
val_stats, val_semantic_pred = experiment.run_experiment(image_typographic_semantic_reg, val_image, val_semantic)

{'RMSE': 0.11760729551315308, 'R2': 0.4122227357775069}


### Linear Regression: Image -> Semantic

In [21]:
image_semantic_reg = linear_model.LinearRegression()

In [22]:
image_semantic_reg.fit(train_image, train_semantic)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [23]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_reg, train_image, train_semantic)

{'RMSE': 1.3475322475642315e-06, 'R2': 0.9999999999189795}


In [24]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_reg, val_image, val_semantic)

{'RMSE': 0.07973702996969223, 'R2': 0.7187900568850076}


In [25]:
np.matmul(image_semantic_reg.coef_, train_image[0]) + image_semantic_reg.intercept_

array([0.33920777, 0.24672401, 0.45607442, 0.60132325, 0.09915429,
       0.37429994, 0.83789253, 0.56135947, 0.11109912, 0.28348565,
       0.4310205 , 0.12420273, 0.33787727, 0.8676631 , 0.90298957,
       0.75516397, 0.91391104, 0.6989753 , 0.53698933, 0.88824755,
       0.4221449 , 0.2785747 , 0.36340952, 0.79731077, 0.2197606 ,
       0.7370262 , 0.55064696, 0.43342263, 0.46513125, 0.59643173,
       0.54990757], dtype=float32)

In [26]:
train_semantic[0]

array([0.33920993, 0.24672677, 0.45607575, 0.60132425, 0.09915713,
       0.37430044, 0.83789067, 0.56135917, 0.11110115, 0.28348806,
       0.43101934, 0.12420486, 0.33788068, 0.86766094, 0.90298933,
       0.75516324, 0.91390994, 0.69897468, 0.5369887 , 0.88824621,
       0.42214719, 0.27857644, 0.36341181, 0.79730803, 0.21976137,
       0.73702373, 0.5506486 , 0.43342463, 0.46512891, 0.59643258,
       0.54990847])

In [27]:
image_semantic_reg.coef_

array([[-1.5516463e-04,  1.0727253e-04, -1.3206899e-04, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-5.5662356e-05,  3.2751821e-05, -9.5323659e-05, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-7.9531455e-05,  5.5499026e-05, -6.2921084e-05, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 4.1108992e-04, -2.8842897e-04,  3.1614350e-04, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-6.8134919e-05,  4.4572633e-05, -8.0040656e-05, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 6.5558590e-05, -4.6466317e-05,  4.6870118e-05, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

### Multilayer Perceptron: Image -> Semantic

In [28]:
image_semantic_nn = neural_network.MLPRegressor(hidden_layer_sizes=(10,), max_iter=500)

In [29]:
image_semantic_nn.fit(train_image, train_semantic)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(10,), learning_rate='constant',
             learning_rate_init=0.001, max_iter=500, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [30]:
train_stats, train_semantic_pred = experiment.run_experiment(image_semantic_nn, train_image, train_semantic)

{'RMSE': 0.16850465536117554, 'R2': -0.674681103926215}


In [31]:
val_stats, val_semantic_pred = experiment.run_experiment(image_semantic_nn, val_image, val_semantic)

{'RMSE': 0.17422597110271454, 'R2': -0.46043920166494035}


#### Example